In [11]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import csv
from dateutil import parser as dateparser

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')

In [12]:
def scrape(url):    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    #print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

In [13]:
def packing(data): 
    if data:
        for r in data['reviews']:
            r["product"] = data["product_title"]
            r['url'] = url
            if 'verified' in r and r['verified'] is not None:
                if 'Verified Purchase' in r['verified']:
                    r['verified'] = 'Yes'
                else:
                    r['verified'] = 'Yes'
            else: 
                    r['verified'] = "No"
            r['rating'] = r['rating'].split(' out of')[0]
            date_posted = r['date'].split('on ')[-1]
            if r['images']:
                r['images'] = "\n".join(r['images'])
            r['date'] = dateparser.parse(date_posted).strftime('%d %b %Y')
            writer.writerow(r)

Formatting guidelines for URL: 
copy and paste url in last page of URL
If last page is single digit, add a space after it, then a new line 
If last page is is two digits, just add a new line 
If last page is more than two digits, will ned to edit the second for loop below.

In [20]:
pages = []
with open('urls.txt', 'r') as urls:
    for url in urls.readlines():
        lastPage = int(url.split('=')[-1]) #extract last page number
        url = url[0:-1]
        for x in range(lastPage, 0, -1):
            #print(url)
            pages.append(url)
            url = url.replace("pageNumber=" + str(x), "pageNumber=" + str(x-1))
#print(pages)
with open('data.csv','a') as outfile, open('urls.txt', 'r') as preurllist:
    writer = csv.DictWriter(outfile, fieldnames=["title","content","date","variant","images","verified","author","rating","product","url"],quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for url in pages: 
        data = scrape(url)
        print(url)
        packing(data)

https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=45
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=44
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=43
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=42
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=41
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=all_reviews&pageNumber=40
https://www.amazon.com/Thick-It-RD26929208-36-OZ/product-reviews/B0011TUM3E/ref=cm

https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=8
https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=7
https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/Thick-Instant-Beverage-Thickener-Original/product-reviews/B00FBP39PC/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&